In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical

def load_file(filepath):
	dataframe = pd.read_csv(filepath, header=None, skiprows=1)
	return dataframe.values

PNES_path ="../pilot/PNES/" # where the folders are saved
ES_path ="../pilot/ES/"

PNESdata = []
ESdata = []

In [2]:
pose_path = os.path.join(PNES_path, 'OpenPose Pose')
hands_path = os.path.join(PNES_path, 'OpenPose Hands')
for index in os.listdir(pose_path):
    pose_csv = os.path.join(pose_path, index)
    hands_csv = os.path.join(hands_path, index)
    if pose_csv.endswith(".csv") or pose_csv.endswith(".CSV"):    
        if hands_csv.endswith(".csv") or hands_csv.endswith(".CSV"):
            pose_values = load_file(pose_csv)
            hands_values = load_file(hands_csv)
            short = min(pose_values.shape[0],hands_values.shape[0])
            dataframe = np.concatenate((pose_values[:short,:],hands_values[:short,:]),1)
            PNESdata.append(dataframe)

In [3]:
pose_path = os.path.join(ES_path, 'OpenPose Pose')
hands_path = os.path.join(ES_path, 'OpenPose Hands')
for index in os.listdir(pose_path):
    pose_csv = os.path.join(pose_path, index)
    hands_csv = os.path.join(hands_path, index)
    if pose_csv.endswith(".csv") or pose_csv.endswith(".CSV"):    
        if hands_csv.endswith(".csv") or hands_csv.endswith(".CSV"):
            pose_values = load_file(pose_csv)
            hands_values = load_file(hands_csv)        
            short = min(pose_values.shape[0],hands_values.shape[0])
            dataframe = np.concatenate((pose_values[:short,:],hands_values[:short,:]),1)
            ESdata.append(dataframe)

In [4]:
PNESarray = np.vstack(PNESdata)
PNESarray = np.reshape(PNESarray, [PNESarray.shape[0], 1, PNESarray.shape[1]])
PNESlabel = np.reshape(np.zeros(PNESarray.shape[0]),[PNESarray.shape[0],1])

ESarray = np.vstack(ESdata)
ESarray = np.reshape(ESarray,[ESarray.shape[0], 1, ESarray.shape[1]])
ESlabel = np.reshape(np.ones(ESarray.shape[0]),[ESarray.shape[0],1])

X = np.concatenate((PNESarray,ESarray),axis=0)
y = np.concatenate((PNESlabel,ESlabel),axis=0)
y = to_categorical(y)

In [5]:
print(PNESarray.shape)
print(ESarray.shape)
print(PNESlabel.shape)
print(ESlabel.shape)
print(X.shape)
print(y.shape)

(48719, 1, 201)
(69111, 1, 201)
(48719, 1)
(69111, 1)
(117830, 1, 201)
(117830, 2)


In [6]:
outfile = "../Datasets/Pilot_Pose+Hands_Frame.npz"
np.savez(outfile, X, y)